# Dynamic Pricing for Urban Parking Lots (Google Colab Version)
### Capstone Project – Summer Analytics 2025
**Consulting & Analytics Club × Pathway**

This notebook first trains and validates pricing models on the static `dataset.csv` and then implements the same logic in a real-time simulation using Pathway.

## 1. Setup & Imports

In [ ]:
# Install required packages
!pip install pathway bokeh pandas numpy panel --quiet

In [ ]:
import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
import datetime
from google.colab import files
from bokeh.palettes import Category10

pn.extension('bokeh')

## 2. Data Loading and Preprocessing

First, we upload and prepare the historical data. This involves cleaning, renaming columns, and engineering features that will be used for both model training and the real-time simulation.

In [ ]:
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

# Rename columns for clarity
df = df.rename(columns={
    'SystemCodeNumber': 'lot_id', 'Capacity': 'capacity', 'Occupancy': 'occupancy',
    'QueueLength': 'queue_length', 'IsSpecialDay': 'is_special_day', 'VehicleType': 'vehicle_type',
    'TrafficConditionNearby': 'traffic_condition', 'LastUpdatedDate': 'date', 'LastUpdatedTime': 'time'
})

# Preprocess and engineer features
df['is_special_day'] = df['is_special_day'].fillna(0).astype(int)
df['queue_length'] = df['queue_length'].fillna(0).astype(int)
df['vehicle_type'] = df['vehicle_type'].fillna('car')
df['traffic_condition'] = df['traffic_condition'].fillna('medium')
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['traffic_level'] = df['traffic_condition'].map(traffic_map).fillna(2)
vehicle_type_map = {'car': 1.0, 'bike': 0.7, 'truck': 1.3}
df['vehicle_type_weight'] = df['vehicle_type'].map(vehicle_type_map).fillna(1.0)
df['occupancy_rate'] = df['occupancy'] / df['capacity']

# Create a clean DataFrame for training and simulation
final_cols = ['lot_id', 'capacity', 'occupancy', 'queue_length', 'is_special_day', 'traffic_level', 'vehicle_type_weight', 'occupancy_rate', 'timestamp']
df_clean = df[final_cols].dropna().sort_values(['lot_id', 'timestamp']).reset_index(drop=True)

print("Cleaned DataFrame ready for modeling:")
print(df_clean.head())

## 3. Model Training & Static Analysis

Here, we define our pricing models and "train" them on the historical data. For these rule-based models, training involves calculating necessary parameters (like min/max demand) and observing model behavior on the static dataset.

In [ ]:
def demand_function(occ_rate, q_len, traffic, special_day, vehicle_weight, alpha=1.5, beta=0.5, gamma=0.3, delta=1.0, epsilon=0.5):
    return (alpha * occ_rate + beta * q_len - gamma * traffic + delta * special_day + epsilon * vehicle_weight)

def demand_based_price(base_price, demand, demand_min, demand_max, lambd=0.7):
    norm_demand = (demand - demand_min) / (demand_max - demand_min + 1e-6)
    price = base_price * (1 + lambd * norm_demand)
    return np.clip(price, 0.5 * base_price, 2.0 * base_price)

def competitive_price(demand_price, occupancy_rate, base_price=10.0):
    price = demand_price
    competitor_prices = np.random.uniform(8, 15, 3)
    min_comp = min(competitor_prices)
    if occupancy_rate >= 0.95 and price > min_comp:
        price = min_comp - 0.5
    return np.clip(price, 0.5 * base_price, 2.0 * base_price)

# Apply models to the static DataFrame to analyze behavior
df_analysis = df_clean.copy()
df_analysis['demand'] = df_analysis.apply(lambda row: demand_function(row['occupancy_rate'], row['queue_length'], row['traffic_level'], row['is_special_day'], row['vehicle_type_weight']), axis=1)

# Calculate normalization stats from the full historical data
demand_stats_static = df_analysis.groupby('lot_id')['demand'].agg(['min', 'max']).rename(columns={'min': 'demand_min', 'max': 'demand_max'})
df_analysis = df_analysis.join(demand_stats_static, on='lot_id')

df_analysis['demand_price'] = df_analysis.apply(lambda row: demand_based_price(10.0, row['demand'], row['demand_min'], row['demand_max']), axis=1)
df_analysis['competitive_price'] = df_analysis.apply(lambda row: competitive_price(row['demand_price'], row['occupancy_rate']), axis=1)

print("Static analysis results:")
print(df_analysis[['lot_id', 'timestamp', 'demand_price', 'competitive_price']].head())

## 4. Real-Time Implementation with Pathway

Now, we deploy the same pricing logic to a real-time stream using Pathway. The functions defined above are reused.

In [ ]:
class ParkingData(pw.Schema):
    lot_id: str
    capacity: int
    occupancy: int
    queue_length: int
    is_special_day: int
    traffic_level: float
    vehicle_type_weight: float
    occupancy_rate: float
    timestamp: datetime.datetime

# Stream the preprocessed data
data_stream = pw.demo.replay_csv(df_clean, schema=ParkingData, input_rate=1000, time_dilation=100)

# Apply demand function to the stream
data_stream = data_stream.with_columns(
    demand=demand_function(
        pw.this.occupancy_rate, pw.this.queue_length, pw.this.traffic_level, 
        pw.this.is_special_day, pw.this.vehicle_type_weight
    )
)

# Calculate streaming demand statistics
demand_stats_stream = data_stream.groupby(pw.this.lot_id).reduce(
    lot_id=pw.this.lot_id,
    demand_min=pw.reducers.min(pw.this.demand),
    demand_max=pw.reducers.max(pw.this.demand)
)

# Join stats and calculate prices on the stream
data_with_demand = data_stream.join(demand_stats_stream, pw.left.lot_id == pw.right.lot_id)
data_with_demand = data_with_demand.with_columns(
    demand_price=demand_based_price(10.0, pw.this.demand, pw.this.demand_min, pw.this.demand_max)
)
final_prices = data_with_demand.with_columns(
    competitive_price=competitive_price(pw.this.demand_price, pw.this.occupancy_rate)
)

results_table = final_prices.select(
    pw.this.lot_id, pw.this.timestamp, pw.this.demand_price, pw.this.competitive_price
)

## 5. Live Dashboard Visualization

This dashboard visualizes the real-time prices calculated by the Pathway simulation.

In [ ]:
def create_price_plot(src, lot_id):
    fig = bokeh.plotting.figure(
        height=300, width=450,
        title=f"Lot {lot_id} Pricing",
        x_axis_type="datetime",
        y_axis_label="Price ($)"
    )
    fig.line('timestamp', 'demand_price', source=src, legend_label='Demand', color=Category10[3][0], line_width=2)
    fig.line('timestamp', 'competitive_price', source=src, legend_label='Competitive', color=Category10[3][1], line_width=2)
    fig.legend.location = 'top_left'
    return fig

lot_ids = df_clean['lot_id'].unique()
grid = pn.GridSpec(sizing_mode='stretch_width')
for i, lot_id in enumerate(lot_ids):
    row, col = divmod(i, 3)
    lot_data = results_table.filter(pw.this.lot_id == lot_id)
    plot = lot_data.plot(lambda src: create_price_plot(src, lot_id), sorting_col='timestamp')
    grid[row, col] = plot

dashboard = pn.Column("## Real-Time Parking Price Dashboard", grid)
dashboard

## 6. Run Pathway Simulation

Run the following cell to start the live simulation. The dashboard above will update in real-time.

In [ ]:
pw.run()